# Gaussian Process 

### Data Training

In [1]:
import sys
import os

# Add the parent directory to the sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import numpy as np
from elliptic_files.train_elliptic import samples_param
from elliptic_files.FEM_Solver import FEMSolver

obs, nthetas = 6, 100
thetas  = samples_param(nthetas,nparam=2)
fem_solver = FEMSolver(np.zeros(2),vert=50)
obs_points = np.linspace(0.2,0.8,obs).reshape(-1,1)
training_data = np.zeros((nthetas,obs ))

for i,theta in enumerate(thetas):
    fem_solver.theta = theta
    fem_solver.solve()
    training_data[i,:] = fem_solver.eval_at_points(obs_points).reshape(1, -1)

/home/s2113174/anaconda3/envs/fenicsx-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import jax.numpy as jnp
from jax.scipy.linalg import solve_triangular
from scipy.optimize import minimize
import jax

jax.config.update("jax_enable_x64", True)
class KernelFunction:
    def __init__(self,kernel_type="squared_exponential"):

        # Supported kernels
        supported_covariances = {
            "squared_exponential": self.squared_exponential_cov,
            "grad_squared_exponential": self.grad_squared_exponential_cov
        }

        if kernel_type in supported_covariances:
            self.covariance = supported_covariances[kernel_type]
        else:
            raise ValueError(f"Kernel type '{kernel_type}' is not supported.")
        
    def euclidean_distance_matrix(self,x, y):
        x_sq = jnp.sum(x ** 2, axis=1, keepdims=True)
        y_sq = jnp.sum(y ** 2, axis=1, keepdims=True).T
        xy = jnp.dot(x, y.T)
        dist_sq = x_sq - 2 * xy + y_sq
        return jnp.sqrt(jnp.maximum(dist_sq, 0.0))

    def squared_exponential_cov(self,r, sigma, l):
        return (sigma**2) * jnp.exp(-0.5 * (r/l) ** 2)

    def grad_squared_exponential_cov(self, r, sigma, l):
        return ((sigma/l)**2) *r * jnp.exp(-0.5 * (r/l) ** 2)
    
    def compute_covariance(self,X,params,Y = None):
        Y = X if Y is None else Y
        d = self.euclidean_distance_matrix(X, Y)
        return self.covariance(d,*params)

class GaussianProcess:
    def __init__(self,X_train,Y_train, x_spc = None,  prior_mean =0, kernel = "squared_exponential"):

        self.X_train = jnp.array(X_train, dtype=jnp.float64)
        self.Y_train = jnp.array(Y_train, dtype=jnp.float64).reshape(-1)
        self.spatial_obs = Y_train.shape[-1]
        self.param_obs = X_train.shape[0]
        self.dim_total = self.spatial_obs*self.param_obs

        self.prior_mean = prior_mean
        self.kernel = KernelFunction(kernel_type=kernel)
        self.opt_params = None  # Store optimized parameters
        self.x_spc = x_spc

    def observed_kernel(self,params, params_spc = None):
        # Step 1: Compute covariance matrix
        spatial_obs_cov = jnp.eye(self.spatial_obs, dtype=jnp.float64) if self.x_spc is None else self.kernel.compute_covariance(self.x_spc,[1,params_spc])
        cov_matrix_ob = self.kernel.compute_covariance(self.X_train,params)
        cov_matrix = jnp.kron(cov_matrix_ob, spatial_obs_cov) +  1e-10 * jnp.eye(self.dim_total , dtype=jnp.float64)
        # Step 2: Cholesky decomposition
        L = jnp.linalg.cholesky(cov_matrix)
        idt = jnp.eye(L.shape[0])
        # Step 3: Solve the triangular system directly
        z = solve_triangular(L, idt, lower=True)
        z_t = solve_triangular(L, idt, lower=True, trans=1)
        return L, jnp.dot(z_t, z)

    def neg_log_likelihood(self, full_params):
        if self.x_spc is not None:
            params_spc = full_params[-1]
            params = full_params[:-1]
        else:
            params = full_params
            params_spc = None
        
        L, cov_inv = self.observed_kernel(params,params_spc)

        # Compute log determinant of K via Cholesky: logdet(K) = 2 * sum(log(diag(L)))
        logdet_K = 2.0 * jnp.sum(jnp.log(jnp.diag(L)))

        return 0.5 * (logdet_K + jnp.dot( self.Y_train.T,jnp.dot(cov_inv, self.Y_train)) + self.dim_total * jnp.log(2 * jnp.pi))

    def nll_grad(self, params):
        """Compute the gradient of the negative log-likelihood"""
        return jax.grad(self.neg_log_likelihood)(params)
    
    def optimize_nll(self,init_params):
        res = minimize(self.neg_log_likelihood,init_params,
                        method="L-BFGS-B",jac=self.nll_grad,bounds=[(1e-5, None)] * len(init_params))
        self.opt_params = res.x  # Store optimized params

        if self.x_spc is not None:
            self.opt_params = res.x[:-1]
            self.opt_params_spc = res.x[-1]
            _, self.obs_cov_inv = self.observed_kernel(self.opt_params, self.opt_params_spc)
            self.spatial_obs_cov = self.kernel.compute_covariance(self.x_spc,[1,self.opt_params_spc])
        else:
            self.opt_params = res.x
            _, self.obs_cov_inv = self.observed_kernel(self.opt_params, None)
            self.spatial_obs_cov = jnp.eye(self.spatial_obs, dtype=jnp.float64)

        self.inv_dif = self.obs_cov_inv @ (self.Y_train - self.prior_mean)
            

    def predict_mean(self, x_test):
        cov_train_test_ind = self.kernel.compute_covariance(x_test,self.opt_params,Y = self.X_train)
        cov_train_test = jnp.kron(cov_train_test_ind, self.spatial_obs_cov)  
        return self.prior_mean + cov_train_test @ self.inv_dif
    
    
    def predict_var(self, x_test):
        cov_test_train_ind = self.kernel.compute_covariance(x_test, self.opt_params, Y=self.X_train)
        cov_test_train = np.kron(cov_test_train_ind, self.spatial_obs_cov)

        #cov_train_test_ind = self.kernel.compute_covariance(self.X_train, self.opt_params, Y=x_test)
        cov_test_ind = self.kernel.compute_covariance(x_test, self.opt_params)
        cov_test = np.kron(cov_test_ind, self.spatial_obs_cov)

        pred_var = cov_test - cov_test_train @ (self.obs_cov_inv @ cov_test_train.T)

        return pred_var


In [3]:
from elliptic_files.FEM_Solver import RootFinder

class Elliptic_PIGP:
    def __init__(self,data_training,lam = 1 /4, kl_terms = 2,kernel = "squared_exponential", l=1,sigma =1):
        # Training data
        self.parameters_data = data_training["parameters_data"]
        self.solutions_data = data_training["solutions_data"]
        self.x_bc = data_training["x_bc"]
        self.y_bc = data_training["y_bc"]
        self.source_func_x = data_training["source_func_x"].reshape(-1,1)
        self.source_func_f_x = data_training["source_func_f_x"]

        # Roots for KL
        self.finder = RootFinder(lam, kl_terms)
        self.roots = jnp.array(self.finder.find_roots())

        # Kernels
        self.kernel_parameter = KernelFunction(kernel_type=kernel)
        self.kernel_spatial = KernelFunction(kernel_type=kernel)
        self.l = l
        self.sigma = sigma

    @property
    def A(self):
        """Compute the A coefficients."""
        return jnp.sqrt(1 / ((1/8)*(5 + (self.roots / 2)**2) + 
                            (jnp.sin(2*self.roots) / (4*self.roots)) * ((self.roots / 4)**2 - 1) - (jnp.cos(2*self.roots)/8)))
    @property
    def an(self):
        """Compute the an values."""
        return jnp.sqrt(8 / (self.roots**2 + 16))
    
    def exp_kl_eval(self,theta, x):
        """
        Eval exp(kl)
        """
        # basis
        basis =self.A[:,None]*self.an[:,None]*(jnp.sin(self.roots[:, None] * x) + \
                                               (self.roots[:, None] / 4) * jnp.cos(self.roots[:, None] * x))
        # Final result: shape (n, m) = (n, r) @ (r, m)
        result = theta @ basis

        return jnp.exp(result)
    
    def grad_kl_eval(self,theta, x):
        """
        kl'
        """
        # basis
        basis =self.A[:,None]*self.an[:,None]*self.roots[:, None]*(jnp.cos(self.roots[:, None] * x) - \
                                               (self.roots[:, None] / 4) * jnp.sin(self.roots[:, None] * x))
        # Final result: shape (n, m) = (n, r) @ (r, m)
        result = theta @ basis

        return result

    def kernel_uf(self,theta,x):
        x = x.reshape(1,-1)

        cov = self.kernel_spatial.compute_covariance(x.reshape(-1,1),Y= self.source_func_x, params=[self.l,self.sigma])
        exp_kl = self.exp_kl_eval(theta,x)
        grad_kl = self.grad_kl_eval(theta,x)

        delta_x = x - self.source_func_x
        print(delta_x.shape,grad_kl.shape,exp_kl.shape)
        pi_gp = ( (- grad_kl * delta_x + 1) / self.l**2 -  delta_x**2  / self.l**4 )* exp_kl
        print(pi_gp.shape)
        return pi_gp*cov


    def kernel_ff(self,theta1,theta2):
        exp_kl1 = self.exp_kl_eval(theta1,self.source_func_x)
        grad_kl1 = self.grad_kl_eval(theta1,self.source_func_x)
        exp_kl2 = self.exp_kl_eval(theta2,self.source_func_x)
        grad_kl2 = self.grad_kl_eval(theta2,self.source_func_x)

        delta_x = (self.source_func_x.reshape(-1,1) - self.source_func_x)

        pi_gp1 = ( (grad_kl1 * delta_x + 1) / self.kernel_spatial.l**2 -  delta_x**2  / self.kernel_spatial.l**4 )* exp_kl1
        pi_gp2 = ( (grad_kl2 * delta_x + 1) / self.kernel_spatial.l**2 -  delta_x**2  / self.kernel_spatial.l**4 )* exp_kl2

        return pi_gp1*pi_gp2*self.kernel_spatial.compute_covariance(self.xf)
    
    def kernel_pde(self, theta1, theta2):
        kuu = self.kernel_parameter.compute_covariance(theta1,theta2)*self.kernel_spatial.compute_covariance(self.x_bc)
        kuf = self.kernel_parameter.compute_covariance(theta1,theta2)*self.kernel_uf(theta2,self.x_bc)
        Kfu = self.kernel_parameter.compute_covariance(theta1,theta2)*self.kernel_uf(theta1, self.x_bc).T
        kff = self.kernel_parameter.compute_covariance(theta1,theta2)*self.kernel_ff(theta1, theta2)

        # Concatenate horizontally: [A | B], [C | D]
        top = jnp.concatenate([kuu, kuf], axis=1)
        bottom = jnp.concatenate([Kfu, kff], axis=1)

        # Concatenate vertically
        kernel_pde = jnp.concatenate([top, bottom], axis=0) +  1e-10 * jnp.eye(self.dim_total , dtype=jnp.float64)

        L = jnp.linalg.cholesky(kernel_pde)
        idt = jnp.eye(L.shape[0])
        # Step 3: Solve the triangular system directly
        z = solve_triangular(L, idt, lower=True)
        z_t = solve_triangular(L, idt, lower=True, trans=1)
        return L, jnp.dot(z_t, z)
    
    def matrix_marginal(self,theta1, theta2, x1,x2):
        Kuu = self.kernel_spatial.compute_covariance(x1, self.xu)
        Kuf = self.kernel_uf(theta2, x1)
        matrix_marginal = jnp.concatenate([Kuu, Kuf], axis=1)

        KuuT = self.kx.cov(x2, self.xu)
        KufT = self.K_uf(theta1, x2)
        matrix_marginal_transpose = jnp.concatenate([KuuT, KufT], axis=1).T
        return matrix_marginal,matrix_marginal_transpose


In [4]:

from scipy.stats import qmc

class LD_cube:
    
    def __init__(self, range_para, dimen = 1):
        
        self.dimen = dimen
        self.range_para = self.para_range_setter(range_para)
        self.points, self.V = self.para_to_points()
        
    def para_range_setter(self, range_para):
        
        if not isinstance(range_para[0],list) and self.dimen == 1:
            range_para = [range_para]
        elif not isinstance(range_para[0],list) and isinstance(self.dimen,int):
            range_para = [range_para] * self.dimen        
        
        return range_para
    
    def para_to_points(self):
                    
        samples =  self.generator()
                
        V = 1
        for j in range(self.dimen):
            samples[:,j] = (self.range_para[j][1]-self.range_para[j][0])*samples[:,j] + self.range_para[j][0]
            V = V * (self.range_para[j][1]-self.range_para[j][0])
        
        
        return samples, V
    
class Halton_cube(LD_cube):

    def __init__(self, range_para, n, dimen = 1):
        
        self.n = n
        super().__init__(range_para, dimen)
    
    def generator(self):
    
        sampler = qmc.Halton(d=self.dimen, scramble=False)
        
        return sampler.random(n=self.n)
    
class data:
    """
    Atrributes:
    k_dm: dimension of unkonwn parameters
    d_tr: number of training points
    n_ob: number of observation points
    
    U: d_tr*k_dm matrix, unknown parameters
    Y: d_tr*n_ob matrix, observational data
    xx: spatial points corresponding to observations
    
    """
    
    def __init__(self, U, Y, k_dm, xx = None):
        
        self.k_dm = k_dm
        self.d_tr = int(U.size/k_dm)
        self.n_ob = int(Y.size/self.d_tr)
        
        self.U = U
        self.Y = Y
        
        self.yy = Y.reshape(-1,1)
        self.xx = xx
        
class data_pde:
    """
    Atrributes:
    k_dm: dimension of unkonwn parameters
    d_tr: number of training points
    n_ob: number of observation points
    
    U: d_tr*k_dm matrix, unknown parameters
    Y: d_tr*n_ob matrix, observational data
    xx: spatial points corresponding to observations
    
    """
    
    def __init__(self, Xbc, ybc, Xf, Yf, k_dm):
        
        self.k_dm = k_dm
        self.d_tr = int(Xf.size/k_dm)
        self.n_ob = int(Yf.size/self.d_tr)
        
        self.Xbc = Xbc
        self.ybc = ybc
        
        self.Xf = Xf
        self.Yf = Yf        
        
        self.yf = Yf.reshape(-1,1)

In [5]:
import numpy as np
from scipy.spatial import distance_matrix


class kernel_function:
    def __init__(self, dimen):
        
        self.dimen = dimen
        self.K = self.cov
    
    def parameter_formalizer(self, para):
        
        para = np.atleast_2d(para).reshape(-1,1)
        
        return para
    
    def input_formalizer(self, x):
        
        return np.atleast_2d(x).reshape(-1,self.dimen)
    
    def weighted_dist_mat(self, l, x1, x2 = None):
        
        if x2 is None:
            x2 = np.copy(x1)
            
        x1 = np.atleast_2d(x1).reshape(-1,self.dimen)
        x2 = np.atleast_2d(x2).reshape(-1,self.dimen)
 
        return distance_matrix(x1/l,x2/l)
 
    def cov(self, x1, x2, n_ob):
        
        raise NotImplementedError(
            'Function cov not implemented. Please initilize a specific kernel function.')
        
    def cov_3d(self, x1, x2, n_ob):
        cov_mat = self.cov(x1,x2)
        a,b = np.shape(cov_mat)
        
        return np.broadcast_to(cov_mat,(n_ob,a,b))
    
    def cov_kron(self, x1, x2, n_ob, Spatial_cov = None):
        
        if Spatial_cov is None:
            Spatial_cov = np.eye(n_ob)
            
        cov_mat = self.cov(x1,x2)
        
        return np.kron(Spatial_cov,cov_mat)
    
class kernel_squared_exponential(kernel_function):
    
    def __init__(self, sigma_square, l, dimen = 1):
        
        self.sigma_square = sigma_square      
        self.l = self.parameter_formalizer(l)  
        self.dimen = dimen
        
        super().__init__(dimen)
        
    def cov(self, x1, x2 = None):
        
        d = self.weighted_dist_mat(self.l, x1, x2)
        
        return self.sigma_square * np.exp(-d**2/2)
    
    def cov_prime(self, x1, x2):
        
        return (-(x1.T-x2.T)/(self.l)**2) * self.cov(x1,x2)


In [6]:
import numpy as np
import jax.numpy as jnp

# Load the data using NumPy
u_Theta = np.loadtxt("2D_trainingset_N10.txt")


# Number of spatial oberservation points
dy = 6
X = np.linspace(0,1,dy)

# Convert to jax.numpy
#u_Theta = jnp.array(data_np)

N = 10
Theta_train = Halton_cube([-1,1], N+1, 2).points[1:,:]

print(Theta_train.shape,u_Theta.shape,X.shape)
tr_data = data(Theta_train, u_Theta.T, 2, X)


(10, 2) (6, 10) (6,)


In [7]:
obs, nthetas = 6, 10
thetas  = samples_param(nthetas,nparam=2)
fem_solver = FEMSolver(np.zeros(2),vert=50)
obs_points = np.linspace(0.2,0.8,obs).reshape(-1,1)
training_data = np.zeros((nthetas,obs ))

for i,theta in enumerate(thetas):
    fem_solver.theta = theta
    fem_solver.solve()
    training_data[i,:] = fem_solver.eval_at_points(obs_points).reshape(1, -1)

In [8]:
training_data.shape

(10, 6)

In [9]:
class Gp_regression:
    def __init__(self, kernel, tr_data, normalizer = 10**(-10), prior_mean = 0):
        
        self.k_dm = tr_data.k_dm
        self.d_tr = tr_data.d_tr
        self.n_ob = tr_data.n_ob
        
        self.mean = prior_mean
        self.k = kernel
        self.tr_data = tr_data
                
        self.normalizer = normalizer
                
        self.K_inv_y, self.K_tr_tr = self.inverse_precomputer()

       
    def inverse_precomputer(self):
        
        K_tr_tr = self.k.cov(self.tr_data.U)   
        K_tr_tr = np.kron(K_tr_tr, np.eye(self.n_ob)) + self.normalizer * np.eye(self.n_ob*self.d_tr)

        K_inv_y = np.linalg.solve(K_tr_tr, self.tr_data.yy - self.mean)
        
        return K_inv_y, K_tr_tr
    
    
    def predict_mean(self, test_u):
        
        K_test_tr = self.k.cov(test_u,self.tr_data.U)
        K_test_tr = np.kron(K_test_tr, np.eye(self.n_ob))#self.extend(K_test_tr)
            
        pred_mean = self.mean + K_test_tr @ self.K_inv_y
                
        return pred_mean
    
    
    def predict_var(self, test_u):
        
        K_test_tr = np.kron(self.k.cov(test_u, self.tr_data.U),np.eye(self.n_ob))
        K_tr_test = np.kron(self.k.cov(self.tr_data.U, test_u),np.eye(self.n_ob)) 
        
        pred_var = np.kron(self.k.cov(test_u), np.eye(self.n_ob))- K_test_tr @ np.linalg.solve(self.K_tr_tr, K_tr_test)
        
        return pred_var
    
    def predict_vars(self, test_u):

        pred_vars = np.zeros((len(test_u),self.n_ob,self.n_ob))
        
        i = 0
        for u in test_u:
            pred_vars[i,:,:] = self.predict_var(u)
            i = i + 1
        
        return pred_vars

    def pred_mean_prime(self, test_u, X = None):
        
        cov_prime = self.k.cov_prime(test_u,self.tr_data.U)
        mean_prime = np.kron(cov_prime, np.eye(self.n_ob)) @ self.K_inv_y
        
        return mean_prime
    
    def pred_var_prime(self, test_u):
        
        pred_var_prime = np.zeros((self.k_dm,self.n_ob,self.n_ob))
        cov_prime = self.k.cov_prime(test_u,self.tr_data.U)
        
        K_tr_test = np.kron(self.k.cov(self.tr_data.U, test_u), np.eye(self.n_ob))
        var_prime = np.kron(cov_prime, np.eye(self.n_ob)) @ np.linalg.solve(self.K_tr_tr, K_tr_test) 
        
        for i in range(self.k_dm):
            pred_var_prime[i,:,:] = var_prime[i::self.k_dm,:]

        return -2 * pred_var_prime
    
    
    
    
class Spc_Gpr(Gp_regression):
    
    def __init__(self, kernel, tr_data, kernel_x, normalizer = 10**(-10), prior_mean = 0):
        
        self.kx = kernel_x
        self.xx = tr_data.xx
        
        super().__init__(kernel, tr_data, normalizer, prior_mean)
        
        
    def inverse_precomputer(self):
        
        K_tr_tr = self.k.cov(self.tr_data.U)         
        K_tr_tr = np.kron(K_tr_tr, self.kx.cov(self.xx)) + self.normalizer * np.eye(self.n_ob*self.d_tr)
        
        K_inv_y = np.linalg.solve(K_tr_tr, self.tr_data.yy - self.mean)
        
        return K_inv_y, K_tr_tr
    
    
    def predict_mean(self, test_u, X = None):
        
        if X is None: X = self.xx
        
        K_test_tr = self.k.cov(test_u,self.tr_data.U)
        K_test_tr = np.kron(K_test_tr, self.kx.cov(X, self.xx) )#self.extend(K_test_tr)
            
        pred_mean = self.mean + K_test_tr @ self.K_inv_y
                
        return pred_mean.reshape(len(X), -1)
    
    
    def predict_var(self, test_u, X = None):
        
        if X is None: X = self.xx
        
        K_test_tr = np.kron(self.k.cov(test_u, self.tr_data.U),self.kx.cov(X, self.xx))
        K_tr_test = np.kron(self.k.cov(self.tr_data.U, test_u),self.kx.cov(self.xx, X)) 
        
        pred_var = np.kron(self.k.cov(test_u), self.kx.cov(X))- K_test_tr @ np.linalg.solve(self.K_tr_tr, K_tr_test)
        
        return pred_var

    def pred_mean_prime(self, test_u, X = None):
        
        if X is None: X = self.xx
        
        cov_prime = self.k.cov_prime(test_u,self.tr_data.U)
        mean_prime = np.kron(cov_prime, self.kx.cov(X, self.xx)) @ self.K_inv_y
        
        
        return mean_prime
    
    def pred_var_prime(self, test_u, X = None):
        
        if X is None: X = self.xx
        
        pred_var_prime = np.zeros((self.k_dm,self.n_ob,self.n_ob))
        cov_prime = self.k.cov_prime(test_u,self.tr_data.U)
        
        K_tr_test = np.kron(self.k.cov(self.tr_data.U, test_u),self.kx.cov(self.xx, X))
        var_prime = np.kron(cov_prime, self.kx.cov(X, self.xx)) @ np.linalg.solve(self.K_tr_tr, K_tr_test) 
        
        for i in range(self.k_dm):
            pred_var_prime[i,:,:] = var_prime[i::self.k_dm,:]

        return -2 * pred_var_prime

class PDE_Gpr(Gp_regression):
    
    def __init__(self, kernel, tr_data, kernel_x, pde_data, normalizer = 10**(-10), prior_mean = 0):
        
        self.kx = kernel_x
        self.xx = tr_data.xx
        
        self.d_f = pde_data.d_tr       
        
        self.xxu = pde_data.Xbc
        self.yu = pde_data.ybc
        
        self.xxf = pde_data.Xf
        self.yf = pde_data.yf
 
        self.yy = tr_data.yy.reshape(tr_data.d_tr,tr_data.n_ob)[:,1:-1].reshape(-1,1)
        super().__init__(kernel, tr_data, normalizer, prior_mean)
        
   #     spc_Gpr = Spc_Gpr(kernel,tr_data,kernel_x)
  #      self.pred_mean_prime = spc_Gpr.pred_mean_prime
 #       self.pred_var_prime = spc_Gpr.pred_var_prime
        
    def inverse_precomputer(self):
        
        K_tr_tr = np.zeros(((self.n_ob-2)*self.d_tr,(self.n_ob-2)*self.d_tr))
        
        for i in range(self.d_tr):
            for j in range(self.d_tr):
                K_tr_tr[i*(self.n_ob-2):(i+1)*(self.n_ob-2),j*(self.n_ob-2):(j+1)*(self.n_ob-2)] = self.Condp_spc(self.tr_data.U[i],self.tr_data.U[j],self.xx[1:-1],self.xx[1:-1]) * self.k.cov(self.tr_data.U[i],self.tr_data.U[j]) 
           
        
        K_tr_tr = K_tr_tr + self.normalizer*np.eye((self.n_ob-2)*self.d_tr)
        
        Condmean_XU = np.zeros((self.d_tr*(self.n_ob-2), 1))        
        for i in range(self.d_tr):
            Condmean_XU[i*(self.n_ob-2):(i+1)*(self.n_ob-2),:] = self.Condp_mean(self.tr_data.U[i],self.xx[1:-1]) 
            
        self.Condmean_XU = Condmean_XU
        K_inv_y = np.linalg.solve(K_tr_tr, (self.yy - Condmean_XU))
        
        return K_inv_y, K_tr_tr
    
    def K_uf(self, u1, X):
        


        lam = 1/4
        theta = np.array([0.38762262, 0.21646897])
        A = np.array([1.05705812, 0.84367913])
        omega = np.array([2.15374797, 4.57785946])
        
        u1 = np.atleast_2d(u1).reshape(1,2)        
        
        def cossin(u, x):
      
            z = 0
            for j in range(2):
                z = z + np.sqrt(theta[j])*u[0][j]*A[j]*(omega[j]*np.cos(omega[j]*x) - lam*omega[j]**2*np.sin(omega[j]*x))

            return z

        def sincos(u, x):

            z = 0
            for j in range(2):
                z = z + np.sqrt(theta[j])*u[0][j]*A[j]*(np.sin(omega[j]*x) + lam*omega[j]*np.cos(omega[j]*x))
                
            return z  
  
        sincos_mat = np.zeros((len(X),self.d_f))
        cossin_mat = np.zeros((len(X),self.d_f))        

        for j in range(self.d_f):
            sincos_mat[:,j] = sincos(u1,self.xxf[j])
            cossin_mat[:,j] = cossin(u1,self.xxf[j])
        
        #print(sincos_mat)
        #print(cossin_mat)
        diff = X.reshape(-1,1)-self.xxf
        #d = self.kx.weighted_dist_mat(1, X, self.xxf)  
        LK = ((self.kx.l**2 - diff**2)*np.exp(sincos_mat) - self.kx.l**2 * diff * cossin_mat * np.exp(sincos_mat))/self.kx.l**4
        mat = LK * self.kx.cov(X,self.xxf)
        
        return mat
    
    def K_ff(self, u1, u2):


        lam = 1/4
        theta = np.array([0.38762262, 0.21646897])
        A = np.array([1.05705812, 0.84367913])
        omega = np.array([2.15374797, 4.57785946])
        
        u1 = np.atleast_2d(u1).reshape(1,2)
        u2 = np.atleast_2d(u2).reshape(1,2)  
        
        def cossin(u, x):
            z = 0
            for j in range(2):
                z = z + np.sqrt(theta[j])*u[0][j]*A[j]*(omega[j]*np.cos(omega[j]*x) - lam*omega[j]**2*np.sin(omega[j]*x))

            return z

        def sincos(u, x):
            z = 0            
            for j in range(2):
                z = z + np.sqrt(theta[j])*u[0][j]*A[j]*(np.sin(omega[j]*x) + lam*omega[j]*np.cos(omega[j]*x))

            return z          
        
        sincos_mat1 = np.zeros((self.d_f,self.d_f))
        sincos_mat2 = np.zeros((self.d_f,self.d_f))
        cossin_mat1 = np.zeros((self.d_f,self.d_f))
        cossin_mat2 = np.zeros((self.d_f,self.d_f))
      

        for j in range(self.d_f):
            sincos_mat1[:,j] = sincos(u1,self.xxf[j])
            cossin_mat1[:,j] = cossin(u1,self.xxf[j])
            sincos_mat2[:,j] = sincos(u2,self.xxf[j])
            cossin_mat2[:,j] = cossin(u2,self.xxf[j])
            
        diff = (self.xxf.reshape(-1,1)-self.xxf)/self.kx.l**2
        #d = self.kx.weighted_dist_mat(self.kx.l**2, self.xxf, self.xxf)  
        LLK = (diff**4 + (cossin_mat2 - cossin_mat1.T)*diff**3 - (6/self.kx.l**2 + cossin_mat1.T*cossin_mat2) *diff**2 + (3*(cossin_mat1.T - cossin_mat2)/self.kx.l**2)*diff + cossin_mat1.T*cossin_mat2/self.kx.l**2 + 3/self.kx.l**4) * np.exp(sincos_mat2) * np.exp(sincos_mat1.T) 
        mat = LLK * self.kx.cov(self.xxf,self.xxf)

        return mat    


    def pde_K(self, u1, u2):
        
        Kuu = self.kx.cov(self.xxu) 
        Kuf = self.K_uf(u2, self.xxu)
        Kfu = self.K_uf(u1, self.xxu).T
        Kff = self.K_ff(u1,u2)
        
        return np.vstack(( np.hstack((Kuu,Kuf)) , np.hstack((Kfu,Kff)) ))

    def Condp_spc(self, u1, u2, x1, x2):
        
        Kuu = self.kx.cov(x1, self.xxu)
        Kuf = self.K_uf(u2, x1)
        qu  = np.hstack((Kuu, Kuf))
        
        Kuu = self.kx.cov(x2, self.xxu)
        Kuf = self.K_uf(u1, x2)
        quT  = np.hstack((Kuu, Kuf)).T
        
        return (self.kx.cov(x1,x2) - qu@np.linalg.solve(self.pde_K(u1,u2),quT)) 
    
    def Condp_mean(self, u1, x1):
        
        Kuu = self.kx.cov(x1, self.xxu)
        Kuf = self.K_uf(u1, x1)
        qu  = np.hstack((Kuu, Kuf))

        return (qu @ np.linalg.solve(self.pde_K(u1,u1), np.vstack((self.yu,self.yf))))  
    
    
    def predict_mean(self, u, x = None):
        
        if x is None: x = self.xx
        
        Condmean_xu = self.Condp_mean(u,x)
                
        KuU = np.zeros((len(x),(self.n_ob-2)*self.d_tr))
        for i in range(self.d_tr):
            KuU[:,i*(self.n_ob-2):(i+1)*(self.n_ob-2)] = self.Condp_spc(u,self.tr_data.U[i],x,self.xx[1:-1])* self.k.cov(u,self.tr_data.U[i])  
           
        
        return Condmean_xu + KuU @ self.K_inv_y
    
    
    def predict_var(self, u, x = None):
        
        if x is None: x = self.xx
        
        Kuu = self.Condp_spc(u,u,x,x) * self.k.cov(u,u)
        
        KuU = np.zeros((len(x),(self.n_ob-2)*self.d_tr))
        for i in range(self.d_tr):
            KuU[:,i*(self.n_ob-2):(i+1)*(self.n_ob-2)] = self.Condp_spc(u,self.tr_data.U[i],x,self.xx[1:-1]) * self.k.cov(u,self.tr_data.U[i])      
        
        return Kuu - KuU @ np.linalg.solve(self.K_tr_tr, KuU.T)  
    
    

    
    
    def pred_mean_prime(self, u, x = None):
        
        if x is None: x = self.xx
            
        KuU = np.zeros((self.k_dm*len(x),(self.n_ob-2)*self.d_tr))        
        for i in range(self.d_tr):
            KuU[:,i*(self.n_ob-2):(i+1)*(self.n_ob-2)] = np.kron(self.k.cov_prime(u.reshape(1,self.k_dm),self.tr_data.U[i].reshape(1,self.k_dm))  ,self.Condp_spc(u,self.tr_data.U[i],x,self.xx[1:-1]))
   
        mean_prime = KuU @ self.K_inv_y
        
        
        return mean_prime
    
    def pred_var_prime(self, u, x = None):
        
        if x is None: x = self.xx
        
        pred_var_prime = np.zeros((self.k_dm,(self.n_ob),(self.n_ob)))
        cov_prime = self.k.cov_prime(u,self.tr_data.U)

        KUu = np.zeros(((self.n_ob-2)*self.d_tr,len(x)))
        for i in range(self.d_tr):
            KUu[i*(self.n_ob-2):(i+1)*(self.n_ob-2),:] = self.Condp_spc(self.tr_data.U[i], u,self.xx[1:-1],x) * self.k.cov(self.tr_data.U[i],u)
            
        KuU = np.zeros((self.k_dm*len(x),(self.n_ob-2)*self.d_tr))        
        for i in range(self.d_tr):
            KuU[:,i*(self.n_ob-2):(i+1)*(self.n_ob-2)] = np.kron(self.k.cov_prime(u.reshape(1,2),self.tr_data.U[i].reshape(1,2))  ,self.Condp_spc(u,self.tr_data.U[i],x,self.xx[1:-1]))
        

        var_prime = KuU @ np.linalg.solve(self.K_tr_tr, KUu) 
        
        for i in range(self.k_dm):
            pred_var_prime[i,:,:] = var_prime[i::self.k_dm,:]

        return -2 * pred_var_prime    
    

In [10]:
tr_data = data(thetas, training_data, 2, obs_points.reshape(-1,))


In [11]:
kernel = kernel_squared_exponential(1,1, dimen = 2)
Gpr = Gp_regression(kernel,tr_data)

def neg_log_marginal_likelihood(hyp):  # ML_GPR

    kernel = kernel_squared_exponential(hyp[0], hyp[1], dimen = 2)
    Gpr = Gp_regression(kernel,tr_data)
    
    K = Gpr.K_tr_tr

    eig_v = np.linalg.eigvals(K)

    L = np.linalg.cholesky(K)
    y = tr_data.yy
    nlml = sum(np.log(np.diag(L))) + 0.5*np.matmul(y.T,np.linalg.solve(K,y))[0] + 0.5*60*np.log(2*np.pi)

    return nlml

neg_log_marginal_likelihood(np.array([1.0, 1.0]))

from scipy.optimize import minimize

hyp = np.array([1,1])
bound = [[10**(-10), 100] for i in range(len(hyp))]
hyp1 = minimize(neg_log_marginal_likelihood, hyp, method = 'L-BFGS-B', bounds = bound)
print("Optimized hyperparameter:",hyp1.x)

kernel = kernel_squared_exponential(*hyp1.x, dimen = 2)
Gpr = Gp_regression(kernel,tr_data)

print(Gpr.predict_mean(np.array([[0.098, 0.430]])))
print(Gpr.predict_var(np.array([[0.098, 0.430]])))

Optimized hyperparameter: [1.57410073 4.23240861]
[[0.42518855]
 [0.66964511]
 [0.92114373]
 [1.18377489]
 [1.45010828]
 [1.69917492]]
[[8.26246152e-07 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 8.26246152e-07 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.26246153e-07 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 8.26246153e-07
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  8.26246153e-07 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 8.26246153e-07]]


In [12]:
def neg_log_marginal_likelihood2(hyp):  # ML_GPR

    kernel = kernel_squared_exponential(hyp[0], hyp[1], dimen = 2)
    kx = kernel_squared_exponential(1, hyp[3], dimen = 1)
    spc_Gpr = Spc_Gpr(kernel,tr_data,kx)
    
    K = spc_Gpr.K_tr_tr
    eig_v = np.linalg.eigvals(K)

    L = np.linalg.cholesky(K)
    y = tr_data.yy
    nlml = 0.5*sum(np.log(np.diag(L))) + 0.5*np.matmul(y.T,np.linalg.solve(K,y))[0] + 0.5*60*np.log(2*np.pi)

    return nlml

hyp = np.array([8,9,1,0.1])
bound = [[10**(-10), 100] for i in range(len(hyp))]
hyp2 = minimize(neg_log_marginal_likelihood2, hyp, method = 'L-BFGS-B', bounds = bound)
print("Optimized hyperparameter:",hyp2.x)

kernel = kernel_squared_exponential(hyp1.x[0],hyp1.x[1], dimen = 2)
kx = kernel_squared_exponential(1, hyp[-1], dimen = 1)
Gpr = Spc_Gpr(kernel,tr_data,kx)

print(Gpr.predict_mean(np.array([[0.098, 0.430]])))
print(Gpr.predict_var(np.array([[0.098, 0.430]])))

Optimized hyperparameter: [7.56931582 5.61604521 1.         0.20870697]
[[0.42518839]
 [0.6696448 ]
 [0.92114354]
 [1.1837749 ]
 [1.45010867]
 [1.69917515]]
[[8.26245905e-07 4.01896176e-07 4.63483886e-08 1.26655065e-09
  8.13775813e-12 3.50279817e-14]
 [4.01896176e-07 8.26245565e-07 4.01896350e-07 4.63483081e-08
  1.26658187e-09 8.13775965e-12]
 [4.63483886e-08 4.01896350e-07 8.26245481e-07 4.01896383e-07
  4.63483080e-08 1.26655065e-09]
 [1.26655066e-09 4.63483081e-08 4.01896382e-07 8.26245483e-07
  4.01896348e-07 4.63483884e-08]
 [8.13775849e-12 1.26658186e-09 4.63483081e-08 4.01896349e-07
  8.26245568e-07 4.01896176e-07]
 [3.50281077e-14 8.13775947e-12 1.26655066e-09 4.63483886e-08
  4.01896178e-07 8.26245906e-07]]


In [13]:
gp = GaussianProcess(thetas,training_data,x_spc=obs_points)

print(gp.neg_log_likelihood(jnp.array([1.0, 1.0,1.0])))

gp.optimize_nll(jnp.array([1.0, 1.0,1.0]))

print(gp.opt_params,gp.opt_params_spc)

print(gp.predict_mean(jnp.array([[0.098, 0.430]])))

print(gp.predict_var(jnp.array([[0.098, 0.430]])))

7464.237732719945
[0.85123893 3.73468095] 0.24955226043524195
[0.42498576 0.66927244 0.92063745 1.18318627 1.44953751 1.69878167]
[[1.13418078e-06 9.00092243e-07 1.29219642e-06 7.91787320e-07
  9.79708166e-07 3.17428682e-07]
 [1.12018215e-06 1.06976342e-06 1.70701853e-06 1.24662449e-06
  1.47410006e-06 5.08108527e-07]
 [9.36357633e-07 1.03739503e-06 1.83274999e-06 1.60349780e-06
  1.82858790e-06 7.07305839e-07]
 [6.78617033e-07 8.16117738e-07 1.57861400e-06 1.67418471e-06
  1.87906303e-06 8.78804086e-07]
 [4.33871908e-07 5.11793338e-07 1.07440841e-06 1.41191036e-06
  1.60469503e-06 9.77894550e-07]
 [2.45111850e-07 2.46522849e-07 5.63864170e-07 9.60645847e-07
  1.13829896e-06 9.54000651e-07]]


In [14]:
Xf = np.linspace(0.,1,10)
print("Observation points for the sourcing function:",Xf)
yf = (4*Xf).reshape(-1,1) # training data
Xbc = np.array([0,1])
ybc = np.array([0,2]).reshape(-1,1)
df = data_pde(Xbc, ybc, Xf, yf, 1)

Observation points for the sourcing function: [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]


In [15]:
def neg_log_marginal_likelihood3(hyp):  # ML_GPR
    print(hyp)
    kernel = kernel_squared_exponential(hyp[0], hyp[1], dimen = 2)
    kx = kernel_squared_exponential(1, hyp[3], dimen = 1)
    pde_Gpr = PDE_Gpr(kernel,tr_data,kx,df,normalizer=10**(-10))
    
    K = pde_Gpr.K_tr_tr
    eig_v = np.linalg.eigvals(K)

    L = np.linalg.cholesky(K)
    y = pde_Gpr.yy
    nlml = 0.5*sum(np.log(np.diag(L))) + 0.5*np.matmul(y.T,np.linalg.solve(K,y))[0] + 0.5*N*np.log(2*np.pi)

    return nlml

Xf = np.linspace(0.,1,10)
print("Observation points for the sourcing function:",Xf)
yf = (4*Xf).reshape(-1,1) # training data
Xbc = np.array([0,1])
ybc = np.array([0,2]).reshape(-1,1)
df = data_pde(Xbc, ybc, Xf, yf, 1)

hyp = hyp2.x
bound = [[10**(-2), 100] for i in range(len(hyp))]
# hyp3 = minimize(neg_log_marginal_likelihood3, hyp, method = 'L-BFGS-B', bounds = bound)

Observation points for the sourcing function: [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]


In [16]:
kernel = kernel_squared_exponential(1, 1, dimen = 2)
kx = kernel_squared_exponential(1, 1, dimen = 1)
pde_Gpr = PDE_Gpr(kernel, tr_data, kx, df)
pde_Gpr.K_uf(pde_Gpr.tr_data.U[0],pde_Gpr.xxu)

array([[ 1.19692243,  1.19835481,  0.80876289,  0.26770657, -0.10837473,
        -0.21351692, -0.11943765,  0.08673703,  0.37427721,  0.74925105],
       [-0.52198305,  0.27218589,  1.04447398,  1.38866671,  1.27227964,
         0.94673965,  0.64357261,  0.46291042,  0.43482834,  0.59519506]])

In [17]:
pde_Gpr.tr_data.U

array([[-0.47849285,  0.7775588 ],
       [-0.94599095, -0.32889673],
       [ 0.84865827,  0.00117257],
       [ 0.22554818, -0.91435519],
       [ 0.08650125, -0.90508749],
       [-0.91447564,  0.6393561 ],
       [ 0.99275248, -0.2209955 ],
       [-0.71292978,  0.41615114],
       [ 0.09094574, -0.50229902],
       [ 0.83499725, -0.94714567]])

In [18]:
pde_Gpr.xxu

array([0, 1])

In [19]:
xf = jnp.linspace(0.,1,10)
yf = (4*Xf).reshape(-1,1) # training data
x_bc = jnp.array([0,1]).reshape(-1,1)
y_bc = jnp.array([0,2]).reshape(-1,1)

data_training = {"parameters_data": thetas,
                 "solutions_data": training_data,
                 "x_bc": x_bc,
                 "y_bc": y_bc,
                 "source_func_x": xf,
                 "source_func_f_x":yf
                   }
elliptic_gp = Elliptic_PIGP(data_training)

In [20]:
data_training["source_func_x"].reshape(-1,1)

Array([[0.        ],
       [0.11111111],
       [0.22222222],
       [0.33333333],
       [0.44444444],
       [0.55555556],
       [0.66666667],
       [0.77777778],
       [0.88888889],
       [1.        ]], dtype=float64)

In [21]:
pde_Gpr.xxu

array([0, 1])

In [22]:
elliptic_gp.kernel_uf(pde_Gpr.tr_data.U,pde_Gpr.xxu)

(10, 2) (10, 2) (10, 2)
(10, 2)


TypeError: mul got incompatible shapes for broadcasting: (10, 2), (2, 10).

In [ ]:
jnp.array([1,2]).reshape(-1,1) - jnp.array([1,2,3])

Array([[ 0, -1, -2],
       [ 1,  0, -1]], dtype=int64)